In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('/Users/bryanzhang/Desktop/career/projects/harmonic_horizons/data/tracks.csv')

In [27]:
df.dropna(axis=0, how='any', inplace=True)
removed_cols = ['Unnamed: 0', 'type', 'uri', 'track_href', 'analysis_url']
df = df.drop(columns=removed_cols)

song_counts = df['name'].value_counts()
duplicate_songs = song_counts[song_counts > 1].index

track_df = df[~df['name'].isin(duplicate_songs)]
track_df = track_df.set_index('name')

In [31]:
mood_distribution = track_df.drop(columns=['id']).groupby('mood').mean()
mood_distribution

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity
mood,,,,,,,,,,,,,,
chill,0.526856,0.352010,5.005653,-17.128099,0.656136,0.068463,0.591460,0.575779,0.144118,0.330904,112.559261,196807.391420,3.864982,48.053874
dark,0.414657,0.326363,5.072808,-16.515951,0.493314,0.052752,0.613608,0.580734,0.147503,0.234963,111.996963,218756.215453,3.751857,36.345716
energetic,0.611131,0.684499,5.464890,-9.199266,0.523834,0.106146,0.217148,0.244188,0.192620,0.439778,123.150639,190883.108150,3.931830,55.306509
epic,0.432082,0.485438,5.128660,-12.340899,0.622680,0.065750,0.462203,0.394649,0.169163,0.329333,116.726178,236497.778969,3.823093,42.041649
happy,0.536919,0.487406,5.297714,-11.831953,0.710792,0.070890,0.467624,0.314374,0.165893,0.433319,115.958064,208707.506645,3.879851,44.335991
romantic,0.606848,0.540329,5.249764,-8.528419,0.586239,0.067519,0.387025,0.103057,0.157122,0.488453,118.063592,225106.008483,3.923657,53.467484
sad,0.518345,0.424231,4.886418,-13.290949,0.637014,0.064768,0.522266,0.349568,0.151950,0.317798,115.221163,229579.320890,3.833995,43.508340
